In [1]:
from PIL import Image
import pandas as pd
import numpy as np
import cv2 as cv
from glob import glob
from sklearn.model_selection import train_test_split
from mitosplit_net import preprocessing, augmentation, evaluation, training, plotting, util
from albumentations import Compose, Rotate, RandomRotate90, HorizontalFlip, Flip, ElasticTransform, GaussNoise, RandomCrop, Resize
from tqdm import tqdm
import os, sys
from os import path
import random as r
from import_augmentation_function import import_fun, aug_fun, import_aug_fun
import matplotlib.pyplot as plt
import tensorflow as tf 

In [2]:
## IMPORTS ALL IMAGES WITHIN THE FOLDER "imager_dir" ##

files_dir = r'C:\Users\roumba\Documents\Software\deep-events\training_data'
images_dir = '220915_mtstaygold_cos7_ZEISS_bf_pos'
joined_path = os.path.join(files_dir, images_dir)
img_size = 256
data_ratio=0.1
data_split_state = None 
number_of_augmentations = 5 

#for some reason, even though it should have been deleted, the first two arrays of axis0 are zeros?# 


In [32]:
## THIS WORKS FOR A NETWORK WITH NO MEMORY ##

all_image_array, all_image_array_gauss = import_fun(joined_path, files_dir, images_dir)
augmentation_data, data_val, augmentation_data_gauss, data_gauss_val =  train_test_split(all_image_array, all_image_array_gauss, 
                                                                                                       test_size=data_ratio, random_state=data_split_state)
data_aug = augmentation_data
data_gauss_aug = augmentation_data_gauss
for j in range(number_of_augmentations):
    augment_data, augment_data_gauss = aug_fun(augmentation_data, augmentation_data_gauss)
    data_aug = np.concatenate((data_aug, augment_data))
    data_gauss_aug = np.concatenate((data_gauss_aug,augment_data_gauss))


100%|██████████| 519/519 [00:00<00:00, 1607.97it/s]


In [8]:
## THIS WORKS FOR MEMORY CONSERVATION ##

all_data_aug, all_data_gauss_aug, all_image_array, all_image_array_gauss = import_aug_fun(joined_path, files_dir, images_dir)
np.shape(all_data_aug)

## DATA SPLIT IN VALIDATION AND AUGMENTATION ##

val_size=int(data_ratio * np.size(all_data_aug,0))
rand_arr= np.zeros(val_size)
aug_size= np.size(all_data_aug,0) - val_size
data_val= np.zeros((val_size, 256,256))
data_gauss_val= np.zeros((val_size, 256,256))
x = list(range(np.size(all_data_aug,0)-val_size))

random_num_row = np.random.choice(x,val_size, replace=False)

for i in range(val_size):
    x= random_num_row[i]
    data_val[i,:,:]= all_image_array[x,:,:]
    data_gauss_val[i,:,:]= all_image_array_gauss[x,:,:]
    if i==0:
        data_aug = np.delete(all_data_aug, x, axis=0)
        data_gauss_aug = np.delete(all_data_gauss_aug, x, axis=0)
    else:
        data_aug = np.delete(data_aug, x, axis=0)
        data_gauss_aug = np.delete(data_gauss_aug, x, axis=0)

100%|██████████| 19/19 [00:00<00:00, 884.37it/s]


KeyboardInterrupt: 

In [38]:
## NORMALIZATION ##    

data_aug_norm = data_aug 
data_gauss_aug_norm= data_gauss_aug
data_val_norm = data_val 
data_gauss_val_norm= data_gauss_val

for framenumber in range(np.size(data_val, 0)):
    # k is the bit of data that we want to set to background and this should be reconsidered since maybe just quoting a number isn't very productive#
    k=0.1 
    kk=1/(1-k)

    # validation data #
    data_vall = (data_val[framenumber])/(np.max(data_val[framenumber]))                                               
    data_vall = data_vall-k
    data_vall[data_vall < 0] = 0   
    data_val_norm[framenumber] = data_vall*kk

    data_gauss_vall = (data_gauss_val[framenumber])/(np.max(data_gauss_val[framenumber])) 
    data_gauss_vall = data_gauss_vall-k
    data_gauss_vall[data_gauss_vall < 0] = 0     
    data_gauss_val_norm[framenumber] = data_gauss_vall*kk                                             
    

for framenumber in range(np.size(data_aug,0)):

    # augmentation data #
    data_augg = (data_aug[framenumber])/(np.max(data_aug[framenumber]))                                               
    data_augg = data_augg-k
    data_augg[data_augg < 0] = 0   
    data_aug_norm[framenumber] = data_augg*kk

    data_gauss_augg = (data_gauss_aug[framenumber])/(np.max(data_gauss_aug[framenumber]))
    data_gauss_augg = data_gauss_augg-k
    data_gauss_augg[data_gauss_augg < 0] = 0     
    data_gauss_aug_norm[framenumber] = data_gauss_augg*kk     

In [47]:
## SHUFFLE ##

r.shuffle(data_val_norm)
r.shuffle(data_gauss_val_norm)
r.shuffle(data_aug_norm)
r.shuffle(data_gauss_aug_norm)

In [71]:
## DEFINE GPU DEVICE WHERE THE CODE WILL RUN ON ##

gpu = tf.config.list_physical_devices('GPU')[0]
print(gpu)
tf.config.experimental.set_memory_growth(gpu, True)
gpu = tf.device('GPU:0/') 

base_dir = r'C:\Users\roumba\Documents\Software\deep-events'
data_path = base_dir+ r'\Norm_data' 
model_path = base_dir+ '\Models'

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [49]:
with gpu:
  nb_filters = 8
  firstConvSize = 9
  batch_size = [8, 16, 32, 256]
  model, history= {}, {}
  
  b=batch_size[1]
  model_name = 'ref_f%i_c%i_b%i'%(nb_filters, firstConvSize, b)
  print('Model:', model_name)
  model[model_name] = training.create_model(nb_filters, firstConvSize)
  history[model_name] = training.train_model(model[model_name], data_aug_norm, data_gauss_aug_norm, b, data_ratio)

Model: ref_f8_c9_b16
* Start Encoder Section *
* Start Center Section *
* Start Decoder Section *
Epoch 1/20
176/176 - 55s - loss: nan - binary_accuracy: 0.9224 - val_loss: nan - val_binary_accuracy: 0.9209 - 55s/epoch - 314ms/step
Epoch 2/20
176/176 - 45s - loss: nan - binary_accuracy: 0.9271 - val_loss: nan - val_binary_accuracy: 0.9209 - 45s/epoch - 255ms/step
Epoch 3/20
176/176 - 45s - loss: nan - binary_accuracy: 0.9271 - val_loss: nan - val_binary_accuracy: 0.9209 - 45s/epoch - 255ms/step
Epoch 4/20
176/176 - 45s - loss: nan - binary_accuracy: 0.9271 - val_loss: nan - val_binary_accuracy: 0.9209 - 45s/epoch - 255ms/step
Epoch 5/20
176/176 - 45s - loss: nan - binary_accuracy: 0.9271 - val_loss: nan - val_binary_accuracy: 0.9209 - 45s/epoch - 256ms/step
Epoch 6/20
176/176 - 45s - loss: nan - binary_accuracy: 0.9271 - val_loss: nan - val_binary_accuracy: 0.9209 - 45s/epoch - 256ms/step
Epoch 7/20
176/176 - 45s - loss: nan - binary_accuracy: 0.9271 - val_loss: nan - val_binary_accura

In [74]:
## SAVE THE DATA ##

folder_name = list(model.keys())

util.save_h5(data_val_norm, data_path, '\data_val')
util.save_h5(data_gauss_val_norm, data_path, '\data_gauss_val')
util.save_h5(data_aug_norm, data_path, '\data_aug')
util.save_h5(data_gauss_aug_norm, data_path, '\data_gauss_aug')




Saving C:\Users\roumba\Documents\Software\deep-events\Norm_data\data_val.h5


ValueError: Unable to create dataset (name already exists)

In [75]:
util.save_model(model, model_path, ['model']*len(model), folder_name)
util.save_pkl(history, model_path, ['history']*len(model), folder_name)

ValueError: 'name' and 'folder_name' lenghts don't match.